# Exercises Neural Networks

In [36]:
import pandas as pd                                                     # Data manipulation

from Resources.Functions import *                                       # Custom functions

## Theoretical questions

### Question 1:
- Bouw een neuraal netwerk op om het resultaat van een student op de gapminder test te bepalen aan de hand van het aantal uren wiskunde in het laatste jaar van het middelbaar, het aantal complotheorieën waarin hij/zij gelooft (zie code vraag 1 vragenlijst les Evaluatimetrieken) en het aantal broers en zussen. Splits de gegevens op is een training en een test data set (`90-10` verhouding). Normaliseer ook de data met behulp van de decimal scaling.

- Opmerking: de variabele gapminder test bevat niet getallen. 
    - Zet de data om naar een geheel getal –integer- dat het percentage weergeeft.
    - Plaats daarna de gegevens in 5 klassen: (0,20) ,…, (80,100].
    - Vervang de klasse door de index van de klasse.


### Question 2:
- Herneem het neuraal netwerk uit vraag 1. Maak de confusion matrix, bereken de meest gebruikte evaluatiemetrieken en teken de ROC.

### Question 3: The Simpsons Revisited
- We willen het geslacht van een Simpson kunnen voorspellen m.b.v. een neuraal netwerk (cfr. hoofdstuk Beslissingsbomen uit Data Science 1). Deze dataset is erg klein en daarom kunnen we geen validatie-set of test-set construeren. 
    - Lees de dataset (‘The Simpsons `original.csv`) van de Simpsons in. 
    - Wat voor soort voorspelling trachten we hier te doen? Regressie of classificatie?
    - Bepaal welke kolommen we niet kunnen gebruiken bij deze voorspelling?
    - Maak nu een input training data set x_simpsons en normaliseer deze (min-max normalisatie). Doe hetzelfde voor de outputkolom(y4_simpsons).
    - Maak een neuraal netwerk met de correcte inputs en outputs. Maak een plot van het ANN.
    - Gebruik je neuraal netwerk om het geslacht van je x_simpsons te voorspellen. Komen ze overeen met de targets? Voorspel ook het geslacht van de onbekende Simpson (Comic guy: `hairlenght=8`, `weight=500`, `age=38`).. Komt dit overeen met de voorspelling die je gemaakt werd bij beslissingsbomen (zie Data Science 1)?

### Question 4: Forecast demo Revisited
- We willen de opbrengsten van een bedrijf per kwartaal van de laatste 5 jaar kunnen voorspellen (cfr. hoofdstuk Forecasting uit Data Science 1). Deze dataset is erg klein en daarom kunnen we geen validatie-set of test-set construeren. 
    - Lees de dataset van de Forecasting in (forecastdemo.csv)
    - Wat voor soort voorspelling trachten we hier te doen? Regressie of classificatie?
    - Bepaal welke kolommen we niet kunnen gebruiken bij deze voorspelling?
    - Maak nu een input training data set x_forcast en een outputkolom (y_forcast). Normaliseer waar nodig.
    - Maak een neuraal netwerk met de correcte inputs en outputs. Maak een plot van de ANN.
    - Gebruik je neuraal netwerk om de opbrengsten te voorspellen die horen bij x_forcast. Komen ze overeen met de werkelijke opbrengsten? Maak één plot van de originele gegevens en de voorspelde gegevens.

### Question 5: Iris
- Laad de dataset iris (`iris.csv`) op en bestudeer de inhoud.
- Wat voor soort voorspelling trachten we hier te doen? Regressie of classificatie?
- Bepaal welke kolommen we niet kunnen gebruiken bij deze voorspelling?
- Maak nu een input data set x_iris en normaliseer deze. Probeer een van de normalisatietechnieken uit die je nog niet hebt gebruikt in een oefening of opdracht.
- Creëer een trainig data set en een test data set (x_train_iris en x_test_iris) die respectievelijk bestaan uit `80%` en `20%` (willekeurig gekozen) van de input data set x_iris.
- Maak een neuraal netwerk met de correcte inputs en outputs.
- Evalueer je neuraal netwerk op je test data set (x_test_iris). Welke nauwkeurigheid (accuracy) behaal je?